In [ ]:
%pip install  
%pip install tabulate sqlite3 requests beautifulsoup4
%pip install 


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for sqlite3


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
def getLinkAndPrice(url):
     
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36 Edg/134.0.0.0",
        "sec-ch-ua-platform": "Windows",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua": "\" Not A;Brand\";v=\"99\", \"Chromium\";v=\"90\", \"Microsoft Edge\";v=\"134\"",
    }
    responseDetails = requests.get(url, headers=headers)
    if responseDetails.status_code == 200:
        soupDetails = BeautifulSoup(responseDetails.content, 'html.parser')
        offer_list = soupDetails.find(id="offer-list-with-pagination")
        ebay_price=0
        price_withShipping=0
        amazon_price=0
        offer_link=""
        shop_name=""
        if offer_list:
            list_items = offer_list.find_all('li')
            for item in list_items:
                offer_items = offer_list.find_all(class_="productOffers-listItem")
                # list was sorted descending
                offer_link = offer_items[0].find("a", class_="productOffers-listItemOfferLink")['href']
                shop_name = offer_items[0].find("a", class_="productOffers-listItemOfferShopV2LogoLink")['data-shop-name']
                price_withShipping = offer_items[0].find("div", class_="productOffers-listItemOfferShippingDetails").text.strip()
                
                ## find out if amazon is available
                for offer_item in offer_items:
                    amazon = "Amazon" in offer_item.find("a", class_="productOffers-listItemOfferShopV2LogoLink")['data-shop-name']
                    ebay = "eBay" in offer_item.find("a", class_="productOffers-listItemOfferShopV2LogoLink")['data-shop-name']
                    if amazon:
                        # amazon_price = offer_item.find("a", class_="productOffers-listItemOfferPrice").text.strip()
                        amazon_price = offer_item.find("div", class_="productOffers-listItemOfferShippingDetails").text.strip()
                        amazon_price= amazon_price.replace("ab", "").replace(".", "").replace(",", ".").replace("€","").replace("inkl Versand","").replace(".\n\n","").replace("siehe Shop","").replace("Zzgl Versandkosten","").strip()
                    if ebay:
                        
                        ebay_price = offer_item.find("div", class_="productOffers-listItemOfferShippingDetails").text.strip()
                        ebay_price= ebay_price.replace("ab", "").replace(".", "").replace(",", ".").replace("€","").replace("inkl Versand","").replace(".\n\n","").replace("siehe Shop","").replace("Zzgl Versandkosten","").strip()
            
                        # ebay_price = offer_item.find("a", class_="productOffers-listItemOfferPrice").text.strip()
            
            price_withShipping= price_withShipping.replace("ab", "").replace(".", "").replace(",", ".").replace("€","").replace("inkl Versand","").replace(".\n\n","").replace("siehe Shop","").replace("Zzgl Versandkosten","").strip()
            return (
                float(price_withShipping) if price_withShipping else 0, 
                float(ebay_price) if ebay_price else 0, 
                float(amazon_price) if amazon_price else 0, 
                offer_link, 
                shop_name
            )


In [ ]:
import sqlite3

def ItemExists(title, price):
    conn = sqlite3.connect('D:\sourcen\Angebots\prices.db')
    cursor = conn.cursor()
    cursor.execute('SELECT price FROM prices WHERE title = ? and price = ?', (title,price))
    existing_price = cursor.fetchone()
    print(existing_price)
    return existing_price is not None

def UpsertDb(title, price_withShipping, amazon_price, amazon_difference, ebay_price, ebay_difference, link):
    elementExsitsInDb= ItemExists(title, price_withShipping)
    conn = sqlite3.connect('D:\sourcen\Angebots\prices.db')
    cursor = conn.cursor()
    
    
    if elementExsitsInDb is False:
        cursor.execute('''
        INSERT INTO prices (title, price, amazon_price, amazon_difference, ebay_price, ebay_difference, link)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (title, price_withShipping, amazon_price, amazon_difference, ebay_price, ebay_difference, link))
    else:
        cursor.execute('''
        UPDATE prices
        SET price = ?, amazon_price = ?, amazon_difference = ?, ebay_price = ?, ebay_difference = ?, link = ?, updated = CURRENT_TIMESTAMP
        WHERE title = ?
        ''', (price_withShipping, amazon_price, amazon_difference, ebay_price, ebay_difference, link, title))

    # Commit the transaction
    conn.commit()
    conn.close()
    return elementExsitsInDb


In [59]:
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate
def getPrices(url):
    data = []
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36 Edg/134.0.0.0",
        "sec-ch-ua-platform": "Windows",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua": "\" Not A;Brand\";v=\"99\", \"Chromium\";v=\"90\", \"Microsoft Edge\";v=\"134\"",
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        print("Status: 200 OK")
        soup = BeautifulSoup(response.content, 'html.parser')
        elements = soup.find_all(attrs={"data-testid": "resultItemLink"})
        for element in elements:
            title = element.find("div", {"data-testid": "productSummary__title"}).text.strip()
            price = element.find("div", {"data-testid": "detailedPriceInfo__price"}).text.strip()
            price = price.replace("ab", "").replace(".", "").replace(",", ".").replace("€","").strip()
            price = float(price)
            print(f"Item: {title} - Price: {price}")
            itemExists= ItemExists(title,price)
            print(f"Item: {title} - Price: {price}-  itemExists: {itemExists}")

            if itemExists == False:
                link = element.find("a", {"data-testid": "link"})['href']
                price_withShipping,ebay_price, amazon_price, offer_link, shop_name = getLinkAndPrice(link)
                if amazon_price==0:
                    amazon_price=price
                if ebay_price==0:
                    ebay_price=price
                amazon_difference= 100 - ((price/amazon_price)*100)
                ebay_difference= 100 - ((price/ebay_price)*100)
                UpsertDb(title, price_withShipping, amazon_price, f"{round(amazon_difference, 2)}%", ebay_price, f"{round(ebay_difference, 2)}%", link);
                data.append([title, price_withShipping, amazon_price, f"{round(amazon_difference, 2)}%", ebay_price, f"{round(ebay_difference, 2)}%", link])
                print(f"Item {title} added to the database")
            else:
                print(f"Item {title} already exists in the database")

            
    else:
        print(f"Fehler: {response.status_code}")
    return data


In [ ]:
import sqlite3


urls=[]
## kopfhörer
urls.append("https://www.idealo.de/preisvergleich/ProductCategory/2520.html")
## festplattn
urls.append("https://www.idealo.de/preisvergleich/ProductCategory/14613.html")
## Lego
urls.append("https://www.idealo.de/preisvergleich/ProductCategory/9552.html")
## Sammelkarten
urls.append("https://www.idealo.de/preisvergleich/ProductCategory/18448.html")
result=[]
headers = ["Title", "Price", "Amazon Preis", "Differenz zu Amazon", "Ebay Preis", "Differenz zu eBay","Link"]
conn = sqlite3.connect('D:\sourcen\Angebots\prices.db')
cursor = conn.cursor()

# Create a table to store the data
cursor.execute('''
CREATE TABLE IF NOT EXISTS prices (
    title TEXT,
    price REAL,
    amazon_price REAL,
    amazon_difference TEXT,
    ebay_price REAL,
    ebay_difference TEXT,
    link TEXT,
    created DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')
conn.close() 
for url in urls:
    # Connect to SQLite database (or create it if it doesn't exist)
    data=getPrices(url)
    # Insert data into the table
   

    # Close the connection
    result.extend(data)
print(tabulate(result, headers=headers, tablefmt="grid"))

Status: 200 OK
Item: Apple AirPods Pro 2 - Price: 218.95
Item: Apple AirPods Pro 2 - Price: 218.95-  itemExists: False
Item Apple AirPods Pro 2 added to the database
Item: Apple AirPods Pro 2 mit USB‑C MagSafe Case - Price: 218.95
Item: Apple AirPods Pro 2 mit USB‑C MagSafe Case - Price: 218.95-  itemExists: False
Item Apple AirPods Pro 2 mit USB‑C MagSafe Case added to the database
Item: Samsung Galaxy Buds 3 Pro SM-R630 - Price: 159.0
Item: Samsung Galaxy Buds 3 Pro SM-R630 - Price: 159.0-  itemExists: True
Item Samsung Galaxy Buds 3 Pro SM-R630 already exists in the database
Item: Apple AirPods Max (2024) - Price: 512.0
Item: Apple AirPods Max (2024) - Price: 512.0-  itemExists: False
Item Apple AirPods Max (2024) added to the database
Item: Sony WH-1000XM4 - Price: 214.0
Item: Sony WH-1000XM4 - Price: 214.0-  itemExists: True
Item Sony WH-1000XM4 already exists in the database
Item: Sennheiser Momentum 4 Wireless - Price: 230.0
Item: Sennheiser Momentum 4 Wireless - Price: 230.0-  